# From Likelihood to Models
### Understanding Uncertainty

## Introduction
- In the last notebook, we introduced maximum likelihood with one or two parameters: Basically, just the random variable and its mean and perhaps variance
- This was a great tool, but not quite what we want for machine learning and data science, where there are typically many observations and many variables
- We want to introduce context into the discussion: features, covariates, controls, etc.
- We are not focusing on machine learning best practices today (train-test split, cross validation, feature engineering, etc.). We're just fitting models and simulating outcomes.

## Introduction
- We're going to use StatsModels to do our maximum likelihood estimation, because it gives us parameter estimates and point estimates quickly; I'm not super fond of StatsModels, but it gets the job done in Python
- The models we look at (linear regression, logistic regression, poisson regression, proportional hazards*) are extremely useful workhorse models, but we're interested in them from a probabilistic perspective: How are we modeling uncertainty, and incorporating covariates into our understanding of prediction?

## Basic Approach
- We have our densities/distributions that yield log-likelihoods, modeling noise/shock/uncertainty/measurement error/etc.
- Those densities/distributions are **parameterized** by variables like $\mu$, $\sigma$, $\lambda$, etc.
- We want to use our data, we want to use our theories, we want to build and control our predictions deliberately
- So 
    1. Pick a parameterized density/distribution with support matching your observed outcomes (e.g. any number, positive number, binary, count, duration, etc.)
    2. Replace the parameters $\mu$, $\sigma$, $\lambda$ by functions of your data. These will often be roughly linear, like $\mu = x_i \beta$ or $\lambda = e^{x_i \beta}$, but your $x_i$'s will typically result from feature engineering. 
    3. Pick the model coefficients/weights ($\beta$) to maximize the likelihood/minimize the loss
    4. Use your model to make predictions (prediction: $\hat{y}$), quantify prediction uncertainty (simulation: $\hat{y} + \sigma \varepsilon$), and quantify model uncertainy (inference: bootstrap the sampling distribution of $\beta$ or $\hat{y}$ or anything else of interest)

## Roadmap
1. Linear and Log-Linear Regression (Predict $ -\infty < \hat{y} < \infty $ or $\hat{y} >0$)
2. Logistic Regression (Predict $ 0 < \hat{y} <1 $)
3. Poisson Regression (Predict a non-negative count $\hat{k} \in \{ 0, 1, 2, ... \}$)

All of these frameworks are just an application of MLE

# 1. Linear Regression

## Linear Regression as MLE
- The workhorse predictive model for everyone who works with data is linear regression: $y_i = x_i \cdot \beta + \sigma \varepsilon_i$, where $\varepsilon_i$ is a random variable with the standard normal distribution
- There are some good reasons why this model is so popular:
    1. It has a closed form solution: $\hat{\beta} = (X^{\top}X)^{-1} X^{\top}y$
    2. The coefficients are reasonably easy to interpret: A $1$% change in $x_{ik}$ leads to a $\beta_k$% change in $y_i$
    3. Even if the true model is non-linear, we can include transformations of the features/control variables to match almost any continuous pattern in the data
    4. If the model is unidentified/over-parameterized, we can use LASSO or Ridge regression to regularize and do model selection
    5. We have hardware that computes inner products like $x_i \cdot \beta$ very efficiently

## Case: Hedonic Pricing
- Using the Ames Price data, predict (log) house prices as a function of amenities/features of the property.

## Deriving the Likelihood
- If we assume that $y_i = x_i \cdot \beta + \sigma \varepsilon_i$, then we can specify the error term as:
$$
\varepsilon_i = \frac{y_i - x_i \cdot \beta}{\sigma}
$$
- If we assume that the error term has the standard normal distribution, $\varepsilon \sim N(0,1)$, then observation $i$'s contribution to the likelihood is:
$$
\dfrac{1}{\sigma} \phi \left( \frac{y_i - x_i \cdot \beta}{\sigma} \right) = \dfrac{1}{\sigma \sqrt{2\pi} } \exp \left( - \frac{1}{2} \left(  \frac{y_i - x_i \cdot \beta}{\sigma}\right)^2 \right) 
$$
- Multiplying all the contributions together yields:
$$
\begin{alignat*}{2}
L(\beta,\sigma) &=& \prod_{i=1}^n \dfrac{1}{\sigma} \phi \left( \frac{y_i - x_i \cdot \beta}{\sigma} \right) \\
&=& \prod_{i=1}^n \dfrac{1}{\sqrt{2\pi} \sigma} \exp \left( - \frac{1}{2} \left(  \frac{y_i - x_i \cdot \beta}{\sigma}\right)^2 \right) 
\end{alignat*}
$$

## The Normal-Linear Log-Likelihood
- And taking logs and simplifying (algebra in the last notebook) gives us:
$$
\ell(\beta, \sigma) = -n \log(\sqrt{2\pi}) - n \log(\sigma) - \sum_{i=1}^n \frac{1}{2} \left( \frac{y_i - x_i \cdot \beta}{\sigma} \right)^2
$$
- This is the same as replacing $\mu = x_i \cdot \beta$ in the simpler log-likelihood we looked at earlier: We're modeling the expected outcome, given $x_i$

## And Notice:
- We can re-arrange a bit as:
$$
\ell(\beta, \sigma) = -n \log(\sqrt{2\pi}) - n \log(\sigma) - \dfrac{1}{2 \sigma^2} \underbrace{ \sum_{i=1}^n \left( y_i - x_i \cdot \beta \right)^2 }_{\text{Sum of Squared Error}}
$$
- In some sense, this approach allows us to break the problem into solving for $\beta$ to get our **point estimates** $\hat{y}_i = x_i \cdot \hat{\beta}$, and then model the distribution around those point estimates
- Whenever you are minimizing $SSE$, you are essentially assuming normally distributed errors, whether you make it explicit or not
- You could make the distributions of the errors more interesting: For example, you could also parameterize and model the variance of the errors, using something like $\sigma_i = e^{x_i \cdot \gamma}$; this is called **heteroskedasticity**


## Exercise: Predictive Densities
Notice that once we have a fitted model, $(\hat{\beta}, \hat{\sigma})$, we can make predictions: 
$$
\hat{y}_i \sim \text{Normal}( \text{mean} = x_i \cdot \hat{\beta}, \text{variance} = \hat{\sigma}^2)
$$

- Make a plot showing the density of the house price prediction, for every house in the dataset.
- For each house $i$, draw 1000 standard normal shocks, and compute a sample of realized values $\hat{y}_i = x_i \cdot \hat{\beta} + \hat{\sigma} \varepsilon_i$. Plot the kernel densities for each property, and compare them with the theoretical distribution. (Hint: From SciPy, import norm, and use norm.rvs(size=1000, random_state=100) to generate your shocks)

## Case:
- Now, let's predict who survives in the breast cancer data set as a function of characteristics, and their distribution of outcomes.

# 2. Logistic Regression as MLE

## Predicting Probabilities
- OLS/Linear regression is great, powerful, simple; not the end of the story, but it's hard to understate how useful it can be
- But it places no restrictions on the values it can predict
- Often we need real probabilities: Values between 0 and 1 that represent the likelihood something happens
- OLS can yield probabilities above 1 and below 0. 
- What's a good alternative?

## Cases: 
- Using the NHANES data, predict who has insurance based on demographic characteristics.
- Using the METABRIC data, predict who survives as a function of characteristics.

## Bernoulli Likelihood/Binary Cross Entropy
- We saw that the log-likelihood for a Bernoulli random variable is
$$
\ell(p) = \sum_{i=1}^N y_i \log(p) + (1-y_i) \log(1-p)
$$
with MLE $\hat{p} = \bar{y}$.
- Logistic regression is built on the Bernoulli distribution, replacing 
$$
p_i = F(x_i \cdot \beta) = \dfrac{1}{1+e^{-x_i \cdot \beta}}
$$
- So we're using the covariates/controls/features $x_i$ to model the probability that $y_i =1$ for each $i$
- This is the simplest neural network (one input layer, zero hidden layers, two nodes in the output layer (0,1)) 

## The Logistic Log-Likelihood
- We then substitute this into the Bernoulli likelihood
$$
\sum_{i=1}^n y_i \log(p_i) + (1-y_i) \log(1-p_i)
$$
to get
\begin{alignat*}{2}
\ell(\beta) &=& \sum_{i=1}^n y_i \log \left( \dfrac{1}{1+e^{-x_i \cdot \beta}} \right) + (1-y_i) \log \left( \dfrac{e^{-x_i \cdot \beta}}{1+e^{-x_i \cdot \beta}}\right) \\
&=& \sum_{i=1}^n y_i \log \left( \dfrac{e^{x_i \cdot \beta}}{1+e^{x_i \cdot \beta}} \right) + (1-y_i) \log \left( \dfrac{1}{1+e^{x_i \cdot \beta}}\right)
\end{alignat*}
- This looks complex, but we've seen how, for logistic regression, $f(z)=F(z)(1-F(z))$, which dramatically simplifies calculations

## Exercise: Interpreting Non-Linear Effects
- For linear regression, a "small" change in variable $x_{ik}$ resulted in a $\beta_k$ change in the prediction $\hat{y}_i$ for every $i$, regardless of the values of the variables: Interpreting how changes in features changed predictions was straightforward, because for a linear model, $\partial \hat{y}/\partial x_{ik} = \beta_k$
- For logistic regression, this interpretation is incorrect, since
$$
p_i = \frac{1}{1+e^{- x_i \cdot \beta}},
$$
and small changes in $x_{ik}$ do not lead to $\beta_k$ changes in $p_i$
- What's the derivative of $p_i$ with respect to $x_{ik}$?
$$
\frac{\partial p_i}{\partial x_{ik}} = \beta_k \dfrac{1}{1+e^{- x_i \cdot \beta}} \dfrac{e^{- x_i \cdot \beta}}{1+e^{- x_i \cdot \beta}} = \beta_k p_i (1-p_i)
$$
- So, on average across the sample, how does a change in variable $k$ impact the predictions? The **average marginal effect (AME)** is
$$
AME_k =  \dfrac{1}{n} \sum_{i=1}^n \beta_k p_i (1-p_i)
$$
- A popular alternative is to compute the probability at the average for each of the variables,
$$
\bar{p} = \frac{1}{1+e^{- \bar{x} \cdot \beta}},
$$
and then compute the **marginal effect at the mean**:
$$
MEM_k = \beta_k \bar{p} (1-\bar{p})
$$
- For the METABRIC survival example, use logistic regression and OLS to predict surival. For the logistic regression model, compute the AME and MEM for your model, and compare the results to the OLS coefficients.

# 3. Poisson Regression as MLE

## Count Data
- One of the motivations for logistic regression was that linear regression might deliver values outside $[0,1]$; i.e. not probabilities
- A similar problem comes up with data that have discrete values, like $\{0, 1, 2, ... \}$; we call this **count data**
- If you are talking about millions of pounds of fish, you can fudge the discreteness of the data, but if you are talking about the number of cancerous lymph nodes detected, you can't
- How do we repurpose MLE for estimating discrete quantities?

## Case Study: 
- Build a model to predict 'Lymph nodes examined positive' and 'Tumor Size' using the METABRIC data, and a distribution of patient outcomes.

## Poisson Regression as MLE
- Which distributions do we have that are useful for count data? The Poisson is the easiest to work with (but the Negative Binomial is another popular one)
- The Poisson mass function is
$$
pr[y_i = k] = \dfrac{\lambda^k e^{-\lambda}}{k!}
$$
where $\lambda >0$, for $k = 0, 1, ...$
- This is for **count data**: Goals scored in a match, number of infected lymph nodes, etc.
- We are fairly forgiving of using OLS with count data, but when it matters, we want to switch to a method like Poisson or Negative Binomial

## Parameterizing $\lambda$
- We want to predict the count $y_i$, given our covariates $x_i$
- The natural parameter to replace is $\lambda$, but it has to be non-negative, or the model blows up
- The standard way to handle this problem is to pick
$$
\lambda_i = e^{x_i \beta}
$$
so that regardless of the $\beta$ we pick, we'll always get a positive $\lambda$


## Deriving the Log-Likelihood
- Let's build the log-likelihood for this case
- The probability that $y_i = k$ is
$$
pr[y_i = k] = \dfrac{(e^{x_i \beta})^k e^{-e^{x_i \beta}} }{k!}
$$
- The log is
$$
\log(pr[y_i=k]) = k \times x_i \cdot \beta - e^{-x_i \beta} - \log(k!)
$$
- And substituting $y_i$ for $k$ yields the contribution of each observation $i$ to the log-likelihood:
$$
y_i \times x_i \cdot \beta - e^{-x_i \beta} - \log(y_i!)
$$
- Summing yields the log-probability of observing the data

## The Poisson Log-Likelihood

- Then our log-likelihood is
$$
\ell(\beta) = \sum_{i=1}^n [ y_i \times x_i \cdot \beta - e^{-x_i \beta} - \log(y_i!)
]
$$
- Again, we don't solve by hand: We take the gradient with respect to $\beta$, and let the computer do gradient descent to get our $\hat{\beta}$ estimator

## Exercise:
- For the METABRIC data, build a model and predict 'Lymph nodes examined positive' for each observation 
- Bootstrap the parameters of your model and plot their sampling densities/distributions
- Bootstrap the densities of your predictions for each observation

## Conclusion
- These are all great entry-level options for modeling and prediction 
    - Linear/log-linear: Real or non-negative reponse values
    - Logistic: Probabilities
    - Poisson: Counts
    - (Proportional Hazards: Predicting duration/arrival/surival times)
- When we say "learning" in machine learning, we literally mean: The model parameters are being updated by way of gradient descent to better fit the data